Processing CHIME FRB180814.J0422+73
=====

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import os
from tqdm import tqdm
import skimage.measure

folder = 'data/CHIME_FRB180814.J0422+73'
# burstdirs = ['burst180814', 'burst180911', 'burst180917', 'burst180919']
# burstdirs = ['burst180814', 'burst180911', 'burst180919']
burstdirs = ['burst180814']

res = {    #(tres, fres)
    'burst180814': (0.983, 16384/400),
    'burst180911': (0.983, 16384/400),
    'burst180917': (1.966, 16384/400),
    'burst180919': (0.983, 16384/400),
}

for burst in burstdirs:
    burstfiles = [s for s in os.listdir(folder +'/'+ burst) if '.npy' in s and 'weights' not in s]
    weightfiles = [s for s in os.listdir(folder +'/'+ burst) if '.npy' in s and 'weights' in s]

    plt.figure(figsize=(20,24))
    plt.figure(figsize=(10,10))

    #plt.suptitle(burst)
    cmap = plt.get_cmap('gray_r')
    cmap.set_bad(color = 'w', alpha = 1.)

    dataStacked = []
    for fname, wfname in zip(burstfiles, weightfiles):
        data    = np.load("{}/{}/{}".format(folder, burst, fname))
        weights = np.load("{}/{}/{}".format(folder, burst, wfname))

        #tl, tr = 425, 575

        data    = data[:, :] 
        weights = weights[:, :]
        weights[13312:14592] = 0
        weights[11776:12032] = 0
        weights[2060:2850]   = 0
        weights[768:1024]    = 0

        np.putmask(data, ~weights.astype(bool), np.nan)
        data = data - 1*data[:, 0:200].mean(axis=1)[:,None]
        
        if dataStacked == []:
            dataStacked = data
        else:
            dataStacked = np.hstack([dataStacked, data])

    downsample = 256
    data = skimage.measure.block_reduce(data, block_size=(downsample, 1), func=np.nanmean)

    plt.subplot(4, 4, burstfiles.index(fname)+1)
    plt.title(burst + ": beam" + fname.split('.')[0].split('beam')[1])
    plt.imshow(data[:,:], origin="lower", aspect="auto", interpolation="nearest", cmap=cmap, extent=(0, res[burst][0], 400, 800))
    plt.colorbar()
    plt.xlabel("Time [s]")
    plt.ylabel("Frequency [MHz]")

    plt.tight_layout()
    #f burst == 'burst180814': plt.savefig('bursts180814.png')
    #if burst == 'burst180814': plt.savefig('bursts180814search.png')

C:\Users\hmsmo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:49: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


MemoryError: Unable to allocate 256. MiB for an array with shape (16384, 4096) and data type float32

<Figure size 1440x1728 with 0 Axes>

<Figure size 720x720 with 0 Axes>